In [1]:
%load_ext autoreload
%autoreload 2
import logging
import os
import sys
import yaml
from pathlib import Path

logging.basicConfig(level=logging.INFO)

try:
    from google.colab import drive
except ImportError:
    logging.info("Local machine detected")
    sys.path.append(os.path.realpath(".."))
else:
    logging.info("Colab detected")
    drive.mount("/content/drive")
    sys.path.append("/content/drive/MyDrive/ecg-reconstruction/src")

from ecg.trainer import Trainer
from ecg.util.path import resolve_path

INFO:root:Local machine detected


2023-07-14 17:42:47.104863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-14 17:42:47.178894: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
def find_best_checkpoint(checkpoint_dir: Path, use_test_set: bool = False) -> Path:
    with open(checkpoint_dir / "trainer_config.yaml", encoding="utf-8") as config_file:
        config = yaml.load(config_file, yaml.Loader)
    config["optimizer"] = None
    config["lr_scheduler"] = None
    if use_test_set:
        eval_set_config = config["dataset"]["eval"]
        eval_set_config["hdf5_filename"] = str(
            Path(eval_set_config["hdf5_filename"]).parent / "test.hdf5"
        )
    losses_and_pth_files: list[tuple[Path, float]] = []
    for pth_file in checkpoint_dir.iterdir():
        logging.info("Evaluating '%s'", pth_file)
        if not pth_file.is_file() or not pth_file.suffix == ".pth":
            continue
        trainer = Trainer(config)
        trainer.load_checkpoint(pth_file)
        trainer.test()
        loss = trainer.metrics.average_loss.get_average()
        rmse = trainer.metrics.rmse.get_average()
        pearson_r = trainer.metrics.pearson_r.get_average()
        logging.info("Loss: %f", loss)
        logging.info("RMSE: %f", rmse)
        logging.info("PearsonR: %f", pearson_r)
        losses_and_pth_files.append((loss, pth_file))
    losses_and_pth_files.sort()
    return losses_and_pth_files[0][1]

In [3]:
find_best_checkpoint(resolve_path("src/checkpoints/Fastformer/20230714-1522"))

INFO:root:Evaluating '/home/yunhao/Documents/ecg-reconstruction/src/checkpoints/Fastformer/20230714-1522/trainer_config.yaml'
INFO:root:Evaluating '/home/yunhao/Documents/ecg-reconstruction/src/checkpoints/Fastformer/20230714-1522/epoch=1_loss=0.01776.pth'
Test 52/52 [0:00:06<0:00:00, 0.119s/it, batch_loss=0.008887, average_loss=0.01776]
INFO:root:Loss: 0.017755
INFO:root:RMSE: 0.133248
INFO:root:PearsonR: 0.892475
INFO:root:Evaluating '/home/yunhao/Documents/ecg-reconstruction/src/checkpoints/Fastformer/20230714-1522/latest'
INFO:root:Evaluating '/home/yunhao/Documents/ecg-reconstruction/src/checkpoints/Fastformer/20230714-1522/best'
INFO:root:Evaluating '/home/yunhao/Documents/ecg-reconstruction/src/checkpoints/Fastformer/20230714-1522/epoch=2_loss=0.01871.pth'
Test 52/52 [0:00:05<0:00:00, 0.0964s/it, batch_loss=0.009623, average_loss=0.01871]
INFO:root:Loss: 0.018712
INFO:root:RMSE: 0.136793
INFO:root:PearsonR: 0.898254
INFO:root:Evaluating '/home/yunhao/Documents/ecg-reconstruction

PosixPath('/home/yunhao/Documents/ecg-reconstruction/src/checkpoints/Fastformer/20230714-1522/epoch=32_loss=0.01493.pth')